In [1]:

# Import the required Libraries
import gradio as gr
import numpy as np
import pandas as pd
import pickle
import transformers

from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from scipy.special import softmax

**App 1**

In [3]:
# Requirements
model_path = "Eva-Gaga/covid-tweet-sentiment-analysis-roberta_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = "@user" if t.startswith("@") and len(t) > 1 else t
        t = "http" if t.startswith("http") else t
        new_text.append(t)
    return " ".join(new_text)

# ---- Function to process the input and return prediction
def sentiment_analysis(text):
    text = preprocess(text)

    encoded_input = tokenizer(text, return_tensors = "pt") # for PyTorch-based models
    output = model(**encoded_input)
    scores_ = output[0][0].detach().numpy()
    scores_ = softmax(scores_)
    
    # Format output dict of scores
    labels = ["Negative", "Neutral", "Positive"]
    scores = {l:float(s) for (l,s) in zip(labels, scores_) }
    
    return scores




# ---- Gradio app interface
app = gr.Interface(fn = sentiment_analysis,
                   inputs = gr.Textbox("Write your text or tweet here..."),
                   outputs = "label",
                   title = "Sentiment Analysis of Tweets on COVID-19 Vaccines",
                   description  = "To vaccinate or not? This app analyzes sentiment of text based on tweets tweets about COVID-19 Vaccines using a fine-tuned roBERTA model",
                   interpretation = "default",
                   examples = [["I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that"]]
                   )

app.launch()

IMPORTANT: You are using gradio version 3.12.0, however version 3.14.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


**App 2**

In [4]:
# Load the fine-tuned model from the HF model hub
model = transformers.AutoModel.from_pretrained("Eva-Gaga/covid-tweet-sentiment-analysis-roberta_model")

# Define a function that takes in input and processes it with the model
def predict(inputs):
    input_ids = transformers.BertTokenizer.from_pretrained("Eva-Gaga/covid-tweet-sentiment-analysis-roberta_model").encode(inputs, return_tensors="pt")
    output = model(input_ids)[0]
    return output

# Create a Gradio interface for the model
app = gr.Interface(fn=predict,
                   inputs=gr.Textbox(prompt="Enter text here:"),
                   outputs=gr.Textbox(prompt= "Predicted sentiment:"),
                   title = "Sentiment Analysis of Tweets on COVID-19 Vaccines",
                   description  = "To vaccinate or not? This app analyzes sentiment of text based on tweets tweets about COVID-19 Vaccines using a fine-tuned roBERTA model"
                   )

# Launch the interface
app.launch()
     

Some weights of the model checkpoint at Eva-Gaga/covid-tweet-sentiment-analysis-roberta_model were not used when initializing RobertaModel: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at Eva-Gaga/covid-tweet-sentiment-analysis-roberta_model and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able

IMPORTANT: You are using gradio version 3.12.0, however version 3.14.0 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Traceback (most recent call last):
  File "c:\Users\selas\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\routes.py", line 292, in run_predict
    output = await app.blocks.process_api(
  File "c:\Users\selas\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1007, in process_api
    result = await self.call_function(fn_index, inputs, iterator, request)
  File "c:\Users\selas\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 848, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\selas\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    ret